In [55]:
import numpy as np 
import cv2
import shutil
import random
import os

In [56]:
def SplitDataBeforeAugmentation(source_folders, test_folder, test_split=0.2):

    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    total_moved = 0

    for folder_path in source_folders:
        class_name = os.path.basename(folder_path)

        class_test_folder = os.path.join(test_folder, class_name)
        if not os.path.exists(class_test_folder):
            os.makedirs(class_test_folder)

        image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith('.jpg')
        ]

        num_test = int(len(image_files) * test_split)
        test_images = random.sample(image_files, num_test)

        for img_name in test_images:
            src_path = os.path.join(folder_path, img_name)

            img = cv2.imread(src_path)
            if img is None:
                os.remove(src_path)
                print(f"{src_path} removed (corrupted)")
                continue

            dst_path = os.path.join(class_test_folder, img_name)
            shutil.move(src_path, dst_path)
            total_moved += 1

        print(f"Moved {num_test} images from {class_name} to test set")

    print(f"\nTotal test images: {total_moved}")


In [57]:
def LoadImagesFromFolder(folder_path):
    """Load all images from a folder"""
    images = []
    image_files = [f for f in os.listdir(folder_path) 
                  if f.lower().endswith('.jpg')]
    
    for img_name in image_files:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    
    return images

In [58]:
def reverseRow(img):
    return img[: ,::-1]

In [59]:
def reverseEntireMatrix(img):
    return img[::-1 , ::-1]

In [60]:
def RotateImage(img , angle = None):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    if angle is None:
        angle = np.random.uniform(-36, 36) 
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)
    return rotated

In [61]:
def ZoomImage(img , zoom_factor = None):
    h, w = img.shape[:2]
    if zoom_factor is None:
        zoom_factor = np.random.uniform(0.8, 0.95) 
    
    nh, nw = int(h * zoom_factor), int(w * zoom_factor)
    if nh >= h or nw >= w:
        return img  
    
    start_h = np.random.randint(0, h - nh + 1)
    start_w = np.random.randint(0, w - nw + 1)
    
    zoomed_img = img[start_h:start_h+nh, start_w:start_w+nw]

    zoomed_img = cv2.resize(zoomed_img, (w, h))
    return zoomed_img

In [62]:
def FlipImageHorizontally(img):
    img = reverseRow(img)
    return img

In [63]:
def FlipImageVertically(img):
    img =  reverseEntireMatrix(img)
    return img

In [64]:
def ChangeBrightness(img, value=None):
    if value is None:
        value = np.random.randint(-20, 20)
    img = img.astype(np.int16) + value
    img = np.clip(img, 0, 255)
    return img.astype(np.uint8)

In [65]:

def ChangeContrast(img, alpha=None):
    if alpha is None:
        alpha = np.random.uniform(0.9, 1.1)
    img = img.astype(np.float32) * alpha
    img = np.clip(img, 0, 255)
    Constrasted_image =  img.astype(np.uint8)
    return Constrasted_image

In [66]:
def ChangeSaturation(img):
    hsv_image = cv2.cvtColor(img , cv2.COLOR_BGR2HSV)
    factor = np.random.randint(0.7, 1.3)
    hsv_image[: , : , 1] = hsv_image[: , : , 1] * factor
    hsv_image[: , : , 1] = np.clip(hsv_image[: , : , 1] , 0 , 255)
    BGR_image = cv2.cvtColor(img , cv2.COLOR_HSV2BGR).astype(np.uint8)
    return BGR_image

In [67]:
def GaussianBlur(img):
    kernel_size = np.random.choice([3 , 5 ,7])
    kernel = (kernel_size , kernel_size)
    Blured_image = cv2.GaussianBlur(img , kernel , 0)
    return Blured_image

In [68]:
def AddGaussianNoise(img):
    mean = 0 
    sigma = np.random.uniform(5 , 15) 
    noise = np.random.normal(mean , sigma , img.shape)
    noisy_image = img + noise 
    noisy_image = np.clip(noisy_image , 0 , 255).astype(np.uint8)
    return noisy_image

In [69]:
def saveImage(img , path , num):
    if img is None or img.size == 0:
        print(f"Warning: image {num} is empty. Skipping save.")
        return
    
    img_resized = cv2.resize(img, (256, 256))
    img_resize = cv2.cvtColor(img_resized , cv2.COLOR_BGR2RGB)
    folder_name = os.path.basename(path)
    cv2.imwrite(f"{path}\\{folder_name}_augImage_{num}.jpg", img_resized)
    print(f"augmented_image {num} saved successfully!")

In [70]:
def ApplyAugumentationTechniques(images , folderPath):
    n = 9 
    cnt = 0
    H, W, C = 256, 256, 3  
    augmented_image = np.zeros((H, W, C))
    for img in images:
      for i in range(n) :
        match i : 
          case 0 : 
            augmented_image = RotateImage(img)  
          case 1 :
            augmented_image = FlipImageHorizontally(img)
          case 2 :
            augmented_image = ChangeBrightness(img)
          case 3:
            augmented_image = ChangeContrast(img)
          case 4 :
            augmented_image = ZoomImage(img)
          case 5 : 
            augmented_image = FlipImageVertically(img)
          case 6:
            augmented_image = ChangeSaturation(img)
          case 7: 
            augmented_image = GaussianBlur(img)
          case 8:
            augmented_image = AddGaussianNoise(img)
        saveImage(augmented_image , folderPath , cnt) 
        cnt = cnt + 1
        
       
        

In [71]:
folderPaths = [
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\cardboard",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\glass",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\metal",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\paper",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\plastic",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\trash"  
      ] 
dataset_path = r"D:\Fourth_level\Semester_1\Machine learning\test_data"
SplitDataBeforeAugmentation(folderPaths , dataset_path)
for folder in folderPaths:
    images = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder,filename)
        if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
          img = cv2.imread(file_path)
          if img is None:
            os.remove(file_path)
            print(f"{file_path} is removed , The image is corrupted and won't open ")
            continue 
          images.append(img)
    ApplyAugumentationTechniques(images , folder)
    
            


D:\Fourth_level\Semester_1\Machine learning\dataset\cardboard\509251d8-4e3a-4f1e-aabc-4d034b0f2455.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\cardboard\2ec9d19b-8027-4c77-a13f-5eee033b9868.jpg removed (corrupted)
Moved 51 images from cardboard to test set
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\90cd02d9-1c1a-4cf4-a08f-100f62bf7968.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\df3b36b0-1e25-4901-a449-a4a3a2c59582.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\393f6b9f-f426-4858-9766-7ba12ed51df3.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\b9304563-57f2-4f71-9f19-d845969f7fbb.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\14f50b32-f879-4c07-ba05-54bf3c6a8693.jpg removed (corrupted)
D:\Fourth_level\Semester_1\Machine learning\dataset\glass\72792053-5f1a-457c-a0da-e5200aadbe1d.jpg removed (corrupte